# 5. 텍스트 생성
## 5.1 일관성 있는 텍스트 생성의 어려움
## 5.2 그리디 서치 디코딩

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

c:\Users\Kang MinJae\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Kang MinJae\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [2]:
import pandas as pd

input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)
        # 첫 번째 배치의 마지막 토큰의 로짓을 선택해 소프트맥스를 적용합니다.
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        # 가장 높은 확률의 토큰을 저장합니다.
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (
                f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f})%"
            )
            iteration[f"Choice {choice_idx + 1}"] = token_choice
        # 예측한 다음 토큰을 입력에 추가합니다.
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)

pd.DataFrame(iterations)

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,most (11.78)%,best (6.66)%,only (5.62)%,first (2.91)%,ultimate (2.23)%
1,Transformers are the most,popular (22.63)%,successful (5.55)%,famous (3.38)%,powerful (3.14)%,important (2.54)%
2,Transformers are the most popular,toys (8.87)%,toy (7.88)%,of (5.03)%,Transformers (4.69)%,franchise (3.88)%
3,Transformers are the most popular toys,of (31.69)%,in (23.73)%,ever (4.85)%,", (4.50)%",for (3.58)%
4,Transformers are the most popular toys of,all (57.47)%,the (21.30)%,2015 (2.34)%,their (1.66)%,2014 (1.54)%
5,Transformers are the most popular toys of all,time (94.71)%,- (1.86)%,. (0.66)%,", (0.56)%",times (0.52)%
6,Transformers are the most popular toys of all ...,. (34.98)%,", (33.86)%",and (7.03)%,! (2.16)%,in (1.73)%
7,Transformers are the most popular toys of all ...,They (10.93)%,\n (9.23)%,The (6.63)%,In (2.91)%,And (2.68)%


In [10]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most popular toys of all time. They


In [12]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \ 
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ 
a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The unicorns were discovered by a team of scientists from the University of Colorado, Boulder, who were studying the effects of climate change on the Andes Mountains.


The team, led by Dr. David R. Smith, discovered the unicorns in the remote valley of the Andes Mountains, in the Andes Mountains of Chile.


The team discovered the unicorns in the


## 5.3 빔 서치 디코딩

In [18]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

In [19]:
def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(
            output.logits[:, :-1, :], labels[:, 1:]
        )
        seq_lob_prob = torch.sum(log_probs[:, input_len:])
    return seq_lob_prob.cpu().numpy()

In [20]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\n로그 확률: {logp:.2f}")

In a shocking finding, scientist discovered \ 
a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The unicorns were discovered by a team of scientists from the University of Colorado, Boulder, who were studying the effects of climate change on the Andes Mountains.


The team, led by Dr. David R. Smith, discovered the unicorns in the remote valley of the Andes Mountains, in the Andes Mountains of Chile.


The team discovered the unicorns in the

로그 확률: -93.94


In [21]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\n로그 확률: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ 
a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


According to the researchers, the unicorns were found in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

The unicorns were found in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke

로그 확률: -28.50


In [22]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, 
                             do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\n로그 확률: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ 
a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


According to a press release from the National Geographic Society, a team of scientists led by University of California, Santa Cruz (UCSC) Professor of Ecology and Evolutionary Biology, Richard Wrangham, discovered the unicorn herd while conducting fieldwork in Peru.

"We were surprised to find that there were so many of them, and that they spoke English," said Professor W

로그 확률: -88.07


## 5.4 샘플링 방법

In [23]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, 
                             temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ 
a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


ColumbbottJJFeQ President Top Not Toocul friendly mammalianSposterium indicum deep rocking Toast tion list Column Exp Save propagate repairs Contact packages CE Richardsctthood circa Haram witchcraft descriptions weapon throw SovietsProof figures play contacting Republicans GiveDoStractical

 improvement elevation breaking morals Fore OrthGood Nah Man Sleeping fists fires innocence fitness growing administration ostcode 820 Zimmerman resign part benevolent possessing standards challenging embassiesigent


In [24]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, 
                             temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ 
a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The unicorns were spotted in the Andes Mountains in Peru, where they have been known to roam for years. The researchers believe that the unicorns are descendants of the Olmecs, a tribe that lived in the area for thousands of years before disappearing.

The researchers have been studying the area for years to find the unicorns, but it's not easy to find the


## 5.5 탑-k 및 뉴클리어스 샘플링

In [25]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, 
                             temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ 
a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The team of researchers from the Universidad de Concepción, in Chile, had been studying the animals on the mountain for several years. They had been trying to find out how they were able to communicate with each other, and were surprised to discover that the unicorns were able to speak English.

The study, published in the journal PNAS, said that the unicorns


In [26]:
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True, 
                             top_p=0.90)
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ 
a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The scientists believe the unicorns came to the valley from a location in the mountains to the west, some 5,200 miles to the east, where there are still some of the largest and most extensive unicorns in the world.

The creatures were born with only a single horn in their bodies, because they were born with their mother's egg, and thus were not able to make


In [27]:
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True, 
                             top_k=50, top_p=0.90)
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ 
a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


"They are talking like a horse in the forest. They are using language we've never heard before and they are behaving like a human," says lead researcher, Dr. Richard Dawkins of Cambridge University. "They are behaving very much like the creatures we thought they would be like."


The creatures, which have yet to be named or described, first appeared on the Internet in November 2007
